# Подключение диска и установка keras.tunes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.0 MB/s eta 0:00:00


# Инициализация

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Dropout, LSTM, Bidirectional, SpatialDropout1D, Flatten, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from tensorflow import keras
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


<ipython-input-3-a7f52d95c376>:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization


In [ ]:
# Максимальное количество слов
num_words = 10000
# Максимальная длина экшена
max_action_len = 50
#Число классов
num_classes = 15

# Загрузка тренировочных данных

In [ ]:
train = pd.read_csv('drive/MyDrive/actionsNPCDataSet/trainActions.csv',header=None,
names=['class', 'text'])

In [ ]:
#Получение экшенов
actions = train['text']

In [ ]:
y_train = utils.to_categorical(train['class'] - 1, num_classes)

# Токенизация

In [ ]:
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
#Обучение токенайзера
tokenizer.fit_on_texts(actions)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(actions)

In [ ]:
x_train = pad_sequences(train_sequences, maxlen=max_action_len)

# Оптимизация гиперпараметров

In [ ]:
def build_model(hp):
    activation_choice = hp.Choice('activation', values=['sigmoid', 'tanh', 'relu', 'elu', 'selu'])
    optimizer_choice = hp.Choice('optimizer', values=['SGD', 'adam', 'adagrad', 'adadelta', 'rmsprop'])
    layers_lstm_num = hp.Int('layers_lstm_num', 0, 2)
    neurons_num = hp.Int('neurons_num', min_value=4, max_value=50, step=8)
    embedding_num = hp.Int('embedding_num', min_value=4, max_value=128, step=16)
    embeddings_regularizer = keras.regularizers.l2(
        hp.Choice('embeddings_regularizer', values=[1e-2, 1e-3, 1e-4], default=1e-3))
    kernel_regularizer = keras.regularizers.l2(hp.Choice('kernel_regularizer', values=[1e-2, 1e-3, 1e-4], default=1e-3))

    model = Sequential()
    model.add(Embedding(input_dim=num_words, output_dim=embedding_num, input_length=max_action_len,
                        embeddings_regularizer=embeddings_regularizer))
    for i in range(layers_lstm_num):
        model.add(LSTM(units=neurons_num, activation=activation_choice, kernel_regularizer=kernel_regularizer,return_sequences=True))
    model.add(LSTM(units=neurons_num, activation=activation_choice, kernel_regularizer=kernel_regularizer))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=optimizer_choice, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=75,
    directory='lstm_actions_dir'
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
activation (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'tanh', 'relu', 'elu', 'selu'], 'ordered': False}
optimizer (Choice)
{'default': 'SGD', 'conditions': [], 'values': ['SGD', 'adam', 'adagrad', 'adadelta', 'rmsprop'], 'ordered': False}
layers_lstm_num (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 2, 'step': 1, 'sampling': 'linear'}
neurons_num (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 50, 'step': 8, 'sampling': 'linear'}
embedding_num (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
embeddings_regularizer (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
kernel_regularizer (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
tuner.search(x_train, y_train, batch_size=128, epochs=5, validation_split=0.2)

Trial 75 Complete [00h 01m 25s]
val_accuracy: 0.9248759746551514

Best val_accuracy So Far: 0.9596031308174133
Total elapsed time: 00h 53m 18s


In [ ]:
tuner.results_summary(num_trials=74)

Results summary
Results in lstm_actions_dir/untitled_project
Showing 74 best trials
Objective(name="val_accuracy", direction="max")

Trial 33 summary
Hyperparameters:
activation: tanh
optimizer: adam
layers_lstm_num: 0
neurons_num: 28
embedding_num: 68
embeddings_regularizer: 0.0001
kernel_regularizer: 0.0001
Score: 0.9596031308174133

Trial 27 summary
Hyperparameters:
activation: tanh
optimizer: adam
layers_lstm_num: 0
neurons_num: 28
embedding_num: 68
embeddings_regularizer: 0.0001
kernel_regularizer: 0.0001
Score: 0.9525159597396851

Trial 36 summary
Hyperparameters:
activation: tanh
optimizer: adam
layers_lstm_num: 0
neurons_num: 28
embedding_num: 68
embeddings_regularizer: 0.0001
kernel_regularizer: 0.0001
Score: 0.9461374878883362

Trial 21 summary
Hyperparameters:
activation: tanh
optimizer: adam
layers_lstm_num: 0
neurons_num: 36
embedding_num: 68
embeddings_regularizer: 0.0001
kernel_regularizer: 0.0001
Score: 0.9454287886619568

Trial 30 summary
Hyperparameters:
activation: t

# Загрузка тестовых данных

In [ ]:
test_actions = pd.read_csv('drive/MyDrive/actionsNPCDataSet/actionsTest.csv',header=None,
names=['class', 'text'])

In [ ]:
#Получение экшенов
test_actions = train['text']

In [ ]:
y_test = utils.to_categorical(train['class'] - 1, num_classes)

# Токенизация

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_actions)

In [ ]:
x_test = pad_sequences(test_sequences, maxlen=max_action_len)

# Проверка лучшей модели

In [ ]:
model = tuner.get_best_models(num_models=1)

In [ ]:
model[0].evaluate(x_test, y_test)

221/221 [==============================] - 2s 4ms/step - loss: 0.4217 - accuracy: 0.9800


[0.4217054545879364, 0.9800028204917908]

# Сохранение модели

In [ ]:
model[0].save('drive/MyDrive/actionsNPCDataSet/actionsLSTMModel.h5')